# Resolve the data to entity-level
The data_cache now has the information we need, on the sentence level. We resolve the entities for each document, and record the sentiment scores for the document and sentence(s) they occur in. We also resolve any sentiment directed towards them at the target level

In [7]:
# venv transform
import json
import os
import time
from collections import Counter, defaultdict
import numpy as np
import random
import pandas as pd
import torch
from helpers import *
from transformers import  pipeline
from tqdm import tqdm


save_root = "outputs/"
tabular_savefolder = os.path.join(save_root, "tabular")
os.makedirs(tabular_savefolder, exist_ok=True)
data_cache = os.path.join(tabular_savefolder, "data_sentencewise.json")
with open(data_cache, encoding = "utf-8") as rf:
    dataset = json.load(rf)
inspect_ids = tuple(set([s["doc_id"] for s in dataset]))

# Resolve entities to document-level

In [33]:
doc_dfs = [] # List of document-entity dataframes to simplify the merging
for doc_id in inspect_ids:
    doc_ents = []
    doc_sents = [s for s in dataset if s["doc_id"] == doc_id]
    ne_groups = [] # List of lists of groups
    for sent in doc_sents:
        for named_e in sent["nes"]:
            # Collect entity with substring match and keep enough data for later
            ne = named_e["text"]
            found = False
            for ne_group in ne_groups:
                if (any([ne in existing for existing in ne_group]) 
                    or any([key in ne for existing in ne_group for key in existing.keys()])):
                    ne_group.append({ne: (named_e.copy(), sent.copy())})
                    found = True
                    break
                else :
                    stripped = ne.rstrip("s").rstrip("'").rstrip()
                    if any([stripped in existing for existing in ne_group]):
                        ne_group.append({ne: (named_e.copy(), sent.copy())})
                        found = True
                        break
            if not found:
                ne_groups.append([{ne: (named_e.copy(), sent.copy())}])
    
    # Now, each named entity document-level is a list of apperances in the text.
    # Next step is to iterate these and make a dataframe
    for e_ent in ne_groups:
        # e_ent is a list of dicts representing each surface form of same entity
        # Find longest text representat
        surface_forms = [key for ent in e_ent for key in ent.keys() ]
        longest = max(surface_forms, key=len)
        if longest.rstrip("s").rstrip("'").rstrip() in surface_forms:
            longest = longest.rstrip("s").rstrip("'").rstrip()
        tsa_sentiments = [value[0]["tsa_pol"] for ent in e_ent for value in ent.values()]
        print(doc_id, longest, tsa_sentiments, surface_forms)




    



dont = ["header", "tokens", "doc_id"]
has_something = [s for s in doc_sents if len(s["tsa_details"]) > 1 and len(s["ner_firsts"]) > 0 ]
has_something = [s for s in doc_sents if len(s["nes"]) > 1  ]
# [print (key, value) for key, value in has_something[1].items() if key not in dont]
print(ne_groups)
pass

602054 Justin Bieber ['Positive'] ['Justin Bieber']
602054 Isac Elliot [''] ['Isac Elliot']
602054 Elliot ['Positive', 'Positive'] ['Elliot', 'Elliot']
003717 Morten Rudå ['', 'Positive'] ['Morten Rudå', 'Morten Rudå']
003717 César Ducasse [''] ['César Ducasse']
003717 Mathieu Peteul [''] ['Mathieu Peteul']
003717 Karl Sundby ['Negative'] ['Karl Sundby']
003717 Kyrre Haugen Sydness ['Positive'] ['Kyrre Haugen Sydness']
003717 Ida Elise Broch ['Positive'] ['Ida Elise Broch']
003717 Sydness ['Negative'] ['Sydness']
109778 Claire Danes ['Positive'] ['Claire Danes']
109778 Carrie Mathison [''] ['Carrie Mathison']
109778 TV 2 [''] ['TV 2']
109778 Danes ['Positive'] ['Danes']
109778 Mandy Patinkin ['Positive'] ['Mandy Patinkin']
109778 Saul Berenson [''] ['Saul Berenson']
109778 Carries [''] ['Carries']
109778 Carrie ['', '', '', ''] ['Carrie', 'Carrie', 'Carrie', 'Carrie']
109778 Nicholas Brody ['', ''] ['Brody', 'Nicholas Brody']
109778 CIA [''] ['CIA']
109778 Abu Nazirs [''] ['Abu Nazirs'